# Tutorial to Work with a JPSS (Polar-Orbiting) VIIRS Level 2 (Granule) Fires Data File

This tutorial was written in December 2022 by Dr. Amy Huff, IMSG at NOAA/NESDIS/STAR (amy.huff@noaa.gov) and Dr. Rebekah Esmaili, STC at NOAA/JPSS (rebekah.esmaili@noaa.gov). It demonstrates how to work with a VIIRS Level 2 (granule) netCDF4 file, including how to handle a netCDF4 file that organizes data variables into **groups** and what aspects to consider for making a beautiful image of point data.

The main steps are:
- Open the file
- Read the global file metadata
    - Recognize when data variables are organized into groups
    - Find names of groups, open a group & read the metadata for variables in the group
- Visualize point satellite data on a map with an image tile background
- Format settings to make a beautiful image:
    - Data markers
    - Figure title
- Save image file

## Import Python packages

We will use five Python packages (libraries) and two Python modules in this Notebook:
- The **xarray** library is used to work with labelled multi-dimensional arrays
- The **netCDF4** library is used to read and write netCDF4 files
- The **NumPy** library is used to perform array operations
- The **Matplotlib** library is used to make plots
- The **Cartopy** library is used to create maps
- The **datetime** module is used to manipulate dates and times
- The **pathlib** module is used to set filesystem paths for the user's operating system

In [1]:
import xarray as xr

from netCDF4 import Dataset

import glob as glob
import numpy as np

import matplotlib as mpl
from matplotlib import pyplot as plt

import cartopy.io.img_tiles as cimgt
from cartopy import crs as ccrs

import datetime

from pathlib import Path

## Set directory path for satellite data files

We set the directory path for the satellite data files using the [**pathlib** module](https://docs.python.org/3/library/pathlib.html#module-pathlib), which automatically uses the correct format for the user's operating system. This helps avoid errors in situations when more than one person is using the same code file, because Windows uses back slashes in directory paths, while MacOS and Linux use forward slashes. 

To keep things simple for this training, we put the satellite data files we downloaded in the current working directory ("cwd"), i.e., the same Jupyter Notebook folder where this code file is located.

In [15]:
directory_path = Path.cwd()

## Topic 3: Understanding the Structure & Contents of netCDF Data Files

### Step 3.1: Open a VIIRS AF I-band satellite data file using xarray & read metadata

Let's use **xarray** to open one of the VIIRS fires data files we downloaded ("file_name"). We set the full path for the data file ("file_id") using **pathlib** syntax.

We open the satellite data file using ```xr.open_dataset``` and then print the file metadata. The contents of a satellite data file are called a "Dataset" in **xarray**, conventionally abbreviated as "ds". 

The global file metadata are listed under "Attributes".

Normally, the metadata for the satellite data in the file are  displayed under "Data variables". But for this file, we see that there are zero (0) "Data variables."  If you encounter this situation with a netCDF4 file using **xarray** it means that the data variables in the file are organized into **groups**.

In [20]:
file_name = 'AF-Iband_v1r0_j01_s202210162118082_e202210162119327_c202210162142235.nc'
file_id = directory_path / file_name

ds = xr.open_dataset(file_id)
ds

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes: (12/65)
    Conventions:                              CF-1.5
    Metadata_Conventions:                     CF-1.5, Unidata Datasset Discov...
    standard_name_vocabulary:                 CF Standard Name Table (version...
    project:                                  NPP Data Exploitation
    institution:                              DOC/NOAA/NESDIS/NDE > Office of...
    naming_authority:                         gov.noaa.nesdis.ncei
    ...                                       ...
    WaterPix:                                 1955108
    MissingPix:                               184319
    GlintPix:                                 0
    CloudPix:                                 2132642
    production_site:                          NSOF
    production_environment:                   OE

### Step 3.2: Open a VIIRS AF I-band satellite data file using netCDF4 & read metadata

We can open groups in a netCDF4 file using **xarray**, but to do that, we need to know the **names** of the groups. Unfortunately, **xarray** can't read the groups metadata, and thus display the names of the groups. This is one of the major shortcomings of using the **xarray** package. 

As a work-around, we have to open the satellite file using either the **netCDF4** Python package, or NASA's free **Panoply** tool. Panoply is incredibly useful for making simple plots to check the contents of satellite data files, so if you are going to be working with satellite data on a regular basis, we recommend [installing Panoply](https://www.star.nesdis.noaa.gov/atmospheric-composition-training/software_panoply_install.php) on your computer.

In this tutorial, we'll use the ```Dataset``` constructor of the **netCDF4** package to open the VIIRS AF I-band file and read the metadata. The names of the groups are listed at the very end, underneath the global file metadata.

In [21]:
root_group = Dataset(file_id, groups='Fire Pixels')
print(root_group)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.5
    Metadata_Conventions: CF-1.5, Unidata Datasset Discovery v1.0
    standard_name_vocabulary: CF Standard Name Table (version 17, 24 March 2011)
    project: NPP Data Exploitation
    institution: DOC/NOAA/NESDIS/NDE > Office of Satellite and Product Operations, NESDIS, NOAA, U.S. Department of Commerce
    naming_authority: gov.noaa.nesdis.ncei
    satellite_name: NOAA-20
    platform: NOAA-20
    instrument_name: VIIRS
    instrument: VIIRS
    title: AF-Iband
    summary: Active Fires EDR: fire mask, qa and list of fire pixels
    history: AF I-Band Version 1.0 (equal to offline vfire375 version 2.5.4)
    processing_level: NOAA Level 2
    cdm_data_type: swath
    creator_name: DOC/NOAA/NESDIS/STAR > AF Algorithm Team, Center for Satellite Applications and Research, NESDIS, NOAA, U.S. Department of Commerce
    creator_email: espcoperations@noaa.gov
    creator_url: http:

### Step 3.3: Print the groups metadata using netCDF4

We can see that there are three groups in this file: "Fire Mask", "Fire Pixels", and "Metadata for OSPO Monitoring Tool". The "Fire Pixels" group sounds like it's what we are looking for - let's print the group metadata to see what data variables are in the group.

In [ ]:
print(root_group.groups['Fire Pixels'])

### Step 3.4: Close the file opened using netCDF4

Now that we know the names of the groups in the file, and we have identified that the fire variables data are in the "Fire Pixels" group, we should close the satellite file we opened using **netCDF4**.

In [ ]:
root_group.close()

### Step 3.5: Open the "Fire Pixels" group using xarray & read group metadata

At this point, we can go back to **xarray** and open the "Fire Pixels" group using the ```group='Fire Pixels'``` argument, and read the metadata for the data variables.

We can see the "Data variables" are one-dimensional arrays with "Dimensions" of 161 (the number of fires detected, "nfire").

"FP_latitude" (fire pixel latitude) and "FP_longitude" (fire pixel longitude) are the variables we will focus on for this example. Click on the data repository icon for these two variables to see the values in the arrays.

In [18]:
ds_group = xr.open_dataset(file_id, group='Fire Pixels')

ds_group

<xarray.Dataset>
Dimensions:                       (nfire: 161)
Dimensions without coordinates: nfire
Data variables: (12/25)
    FP_line                       (nfire) uint16 ...
    FP_sample                     (nfire) uint16 ...
    FP_latitude                   (nfire) float32 ...
    FP_longitude                  (nfire) float32 ...
    FP_T4                         (nfire) float32 ...
    FP_T5                         (nfire) float32 ...
    ...                            ...
    FP_day                        (nfire) uint8 ...
    FP_SolZenAng                  (nfire) float32 ...
    FP_SolAzAng                   (nfire) float32 ...
    FP_ViewZenAng                 (nfire) float32 ...
    FP_ViewAzAng                  (nfire) float32 ...
    FP_PersistentAnomalyCategory  (nfire) uint8 ...

## Topic 4: Handling Data Arrays

Many level 2 products consist of smaller, 2 minute granules of data. For this workshop we downloaded three VIIRS Active Fire I-band files. We can use the glob command to list all files in our working directory that match the format 'AF-Iband*' where the * is a wildcard for any number of characters>

In [39]:
files = glob.glob('AF-Iband*')
files

['AF-Iband_v1r0_j01_s202210162118082_e202210162119327_c202210162142235.nc',
 'AF-Iband_v1r0_j01_s202210162116424_e202210162118070_c202210162142397.nc',
 'AF-Iband_v1r0_j01_s202210162119340_e202210162120567_c202210162143123.nc']

xarray has a convenient function called ```open_mfdataset()``` to open 2 or more files at once **as long as they contain the same kind of data**. If the file does not contain explicit matching coordinates, you'll generally need the following two options:

* ```combine```: the default is by coordinates ("by_coords") but you can also use "nested" and provide a specific dimension by name
* ```concat_dim```: the dimension name across which you want to combine the data

It usually helps to first open and study a single file before using the open_mfdataset. Also, since there are groups this particular product, we'll need to pass that option in like we did for ```xr.open_dataset()```.

In [40]:
open_kwargs = dict(groups='Fire Pixels')
ds_2 = xr.open_mfdataset(files, engine='netcdf4', concat_dim='nfire',  combine='nested', group='Fire Pixels')
ds_2

<xarray.Dataset>
Dimensions:                       (nfire: 439)
Dimensions without coordinates: nfire
Data variables: (12/25)
    FP_line                       (nfire) uint16 dask.array<chunksize=(161,), meta=np.ndarray>
    FP_sample                     (nfire) uint16 dask.array<chunksize=(161,), meta=np.ndarray>
    FP_latitude                   (nfire) float32 dask.array<chunksize=(161,), meta=np.ndarray>
    FP_longitude                  (nfire) float32 dask.array<chunksize=(161,), meta=np.ndarray>
    FP_T4                         (nfire) float32 dask.array<chunksize=(161,), meta=np.ndarray>
    FP_T5                         (nfire) float32 dask.array<chunksize=(161,), meta=np.ndarray>
    ...                            ...
    FP_day                        (nfire) uint8 dask.array<chunksize=(161,), meta=np.ndarray>
    FP_SolZenAng                  (nfire) float32 dask.array<chunksize=(161,), meta=np.ndarray>
    FP_SolAzAng                   (nfire) float32 dask.array<chunksize=(161,), meta=np.ndarray>
    FP_ViewZenAng                 (nfire) float32 dask.array<chunksize=(161,), meta=np.ndarray>
    FP_ViewAzAng                  (nfire) float32 dask.array<chunksize=(161,), meta=np.ndarray>
    FP_PersistentAnomalyCategory  (nfire) uint8 dask.array<chunksize=(161,), meta=np.ndarray>

## Topic 5: Making Composite (RGB) Images 

This topic is not applicable to the VIIRS AF I-band data files. We can only make RGB images from satellite data files that contain sensor band radiances or brightness temperature variables.

## Topic 6: Working with Map Projections

### Step 6.1: Make a quick scatter plot of Fire Pixel latitude vs. longitude using xarray

When you're working with a new data product, it's good practice to make a quick plot to check the distribution of data in the file. We can do this using the plotting functions built into **xarray**.

Overview of **xarray** plotting functions: https://docs.xarray.dev/en/stable/user-guide/plotting.html

We use the arguments for the **xarray** scatter plot function to plot the fires as red square markers.

For our map, we will be zooming in on the Nakia Creek Fire and the Siouxon and Sunset Fires in southern Washington state, near the border with Oregon: that is the cluster of data in the upper left side of the plot.

In [ ]:
ds_group.plot.scatter(x='FP_longitude', y='FP_latitude', color='red', marker='s')

### Step 6.2: Add an image tile map background using Cartopy

When plotting point data, like satellite fire detections, an image map background provides geographic context for the data. **Cartopy** has many built-in interfaces for adding static images and map backgrounds.

**Cartopy** image tiles interfaces: https://scitools.org.uk/cartopy/docs/latest/reference/io.html#image-tiles

I've tested many of these options, and I like the ESRI World Street Map tiles, imported using **Cartopy's** web tile retrieval interface, which shows both political and geographic map features.

Let's generate the ESRI World Street Map background for the region near the Washington/Oregon border where the Nakia Creek Fire and the Siouxon and Sunset Fires were burning. We'll leave out the satellite fire data for now, to see the map clearly.

First we set the URL for the ESRI World Street Map tiles and then load the tiles ("map_background") using **Cartopy's** interface: "cimgt.GoogleTiles(url=esri_tiles)". Then we set up a figure in Matplotlib and add "geoaxes" with a map projection using **Cartopy**; for best results, we set the **map projection** for the figure to be the same as that of the ESRI map background (which happens to be **Mercator**). 

We add the map_background using "add_image". The "zoom_level" is the resolution of ESRI map background tile; the higher the "zoom_level", the greater the map detail. You can change the zoom level to see how the map resolution changes.

We we zoom into our region of interest along the Washington/Oregon border using "set_extent([map_corners])" where the "map_corners" are the [western_longitude, eastern_longitude, southern_latitude, northern_latitude] of the zoomed-in map in degrees (use negative values to indicate °S latitude or °W longitude). Specifying the Plate Carree map projection using the argument "crs=ccrs.PlateCarree()" tells Cartopy that the map_corners are entered in geographic coordinates (latitude and longitude).

In [ ]:
esri_tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}.jpg'
map_background = cimgt.GoogleTiles(url=esri_tiles)

fig = plt.figure(figsize=(10, 10))

ax = plt.axes(projection=map_background.crs)

zoom_level = 11
ax.add_image(map_background, zoom_level)

ax.set_extent([-122.75, -122.0, 45.5, 46.0], crs=ccrs.PlateCarree())

plt.show()

### Exercise Fire-1: CHANGE ZOOM LEVEL FOR IMAGE TILE MAP BACKGROUND

In the code block below, fill in the missing "zoom_level" for the image tile map background, and generate the image. Try a few different zoom levels to see how they affect the details included on the map image.

In [ ]:
esri_tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}.jpg'
map_background = cimgt.GoogleTiles(url=esri_tiles)

fig = plt.figure(figsize=(10, 10))

ax = plt.axes(projection=map_background.crs)

# SET ZOOM LEVEL
zoom_level = 
ax.add_image(map_background, zoom_level)

ax.set_extent([-122.75, -122.0, 45.5, 46.0], crs=ccrs.PlateCarree())

plt.show()

## Topic 7: Adding Professional Touches to Images

### Step 7.1: Plot fire detections on a map background using Matplotlib & Cartopy

Now that we know how to import a map background image, we can plot the satellite fire detections on top of the map image. 

We plot the fire data using **Matplotlib's** simple "plot" plotting function. Note **xarray** makes it easy to plot Data Arrays using "xarray.DataArray", e.g. "ds_group.FP_longitude", "ds_group.FP_latitude".

We customize the appearance of the fire detection markers by making them small red filled squares with a thin black edge.

The plotting function argument "transform=ccrs.PlateCarree()" tells **Cartopy** that the fire data are in geographic coordinates (lat/lon). This argument must be included when plotting satellite data that are in geographic coordinates, or the data will not plot correctly on the map projection.

We also add an automatically generated plot title by extracting information from the data file name, including the date and observation time in UTC.

In [ ]:
esri_tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}.jpg'
map_background = cimgt.GoogleTiles(url=esri_tiles)

fig = plt.figure(figsize=(10, 10))

ax = plt.axes(projection=map_background.crs)

zoom_level = 11
ax.add_image(map_background, zoom_level)

ax.set_extent([-122.75, -122.0, 45.5, 46.0], crs=ccrs.PlateCarree())

ax.plot(ds_group.FP_longitude, ds_group.FP_latitude, color='red', marker='s', linewidth=0, mec='k', mew=0.2, ms=3, 
        transform=ccrs.PlateCarree())

date = datetime.datetime.strptime(file_id.name.split('_')[-3][1:9], '%Y%m%d').date()
date = date.strftime('%d %b %Y')
title = 'NOAA-20/VIIRS Active Fires I-band  ' + date + ' ' + file_id.name.split('_')[-3][9:11] + ':' + file_id.name.split('_')[-3][11:13] + ' UTC'
plt.title(title, pad=10, size=8, weight='bold')

plt.show()

### Exercise Fire-2: SET YOUR OWN MARKERS FOR FIRE DETECTIONS

Use **Matplotlib** to set your own style and color for the fire markers. In the code block below, fill in the missing "color", "marker", "mec", "mew", and "ms" arguments in the "ax.plot" command, and generate the image. Try a few different marker/edge color combinations.

Note: Keep the "linewidth=0" or **Matplotlib** will draw a line to connect the fire markers. 

**Matplotlib** kwargs for plot appearance (Line2D): https://matplotlib.org/stable/api/_as_gen/matplotlib.lines.Line2D.html#matplotlib.lines.Line2D

In [ ]:
esri_tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}.jpg'
map_background = cimgt.GoogleTiles(url=esri_tiles)

fig = plt.figure(figsize=(10, 10))

ax = plt.axes(projection=map_background.crs)

zoom_level = 11
ax.add_image(map_background, zoom_level)

ax.set_extent([-122.75, -122.0, 45.5, 46.0], crs=ccrs.PlateCarree())

## SET FIRE MARKER KWARGS: "color=", "marker=", "mec=", "mew=", "ms="
ax.plot(ds_group.FP_longitude, ds_group.FP_latitude, color=, marker=, linewidth=0, mec=, mew=, ms=, 
        transform=ccrs.PlateCarree())

date = datetime.datetime.strptime(file_id.name.split('_')[-3][1:9], '%Y%m%d').date()
date = date.strftime('%d %b %Y')
title = 'NOAA-20/VIIRS Active Fires I-band  ' + date + ' ' + file_id.name.split('_')[-3][9:11] + ':' + file_id.name.split('_')[-3][11:13] + ' UTC'
plt.title(title, pad=10, size=8, weight='bold')

plt.show()

### Step 7.2: Save the map of fire detections as an image file using Matplotlib

Now that we have created our map of fire detections, we need to save the image file so we can use the image in a research paper, presentation, website, or social media.

We can save the map as an image file using **Matplotlib's** "fig.savefig" function. 

* **Matplotlib** "savefig" settings: https://matplotlib.org/stable/api/figure_api.html#matplotlib.figure.Figure.savefig

We can extract information from the satellite data file name to automatically generate a file name for the saved file, including the observation date and time.

We can change the resolution of the saved image file using the "dpi=" argument in "fig.savefig()". The higher the dpi, the higher the figure resolution, but the larger the file size and the longer it will take to save the file. For figures that include an image map background, like this one, set a very high dpi, such as dpi=1000, so the details of the map background will be clear.

We can also set the "facecolor" (background color) for the plot, and I like to set "bbox_inches='tight'" to minimize the bounding box around the figure (to zoom in "tight" on the plot).

Note that we can also specify the format for the saved file ("saved_file_format"). The default is a .png file, but **Matplotlib** has many options. Try saving your image file with a different format, such as .jpg or .pdf, to see the differences.

**Matplotlib** saved file format options: .eps, .jpeg, .jpg, .pdf, .pgf, .png, .ps, .raw, .rgba, .svg, .svgz, .tif, .tiff

By default, the image file is saved to the "current working directory", where this Notebook file is located.

**This is the final step! Comments are included in the code below, to notate each step in the process of generating a beautiful image of VIIRS Level 2 point satellite data.**

In [ ]:
# Get ESRI World Street Map tiles using Cartopy's web tile retrieval
esri_tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}.jpg'
map_background = cimgt.GoogleTiles(url=esri_tiles)

# Set up figure in Matplotlib
fig = plt.figure(figsize=(10, 10))

# Add axes to figure and set map projection to be same as ESRI World Street Map tiles (Mercator)
ax = plt.axes(projection=map_background.crs)

# Plot ESRI World Street Map as background at specified resolution
zoom_level = 11
ax.add_image(map_background, zoom_level)

# Set extent of map to zoom-in to area of interest
# Use negative values to indicate °W longitude, e.g., 100 °W = -100
# Use negative values to indicate °S latitude, e.g., 30 °S = -30
ax.set_extent([-122.75, -122.0, 45.5, 46.0], crs=ccrs.PlateCarree())

# Plot fire detections using small filled red square markers with thin black border
# "transform=ccrs.PlateCarree()" argument is required b/c data are in geographic coordinates
ax.plot(ds_group.FP_longitude, ds_group.FP_latitude, color='red', marker='s', linewidth=0, mec='k', mew=0.2, ms=3, 
        transform=ccrs.PlateCarree())

# Create plot title automatically using information from file name
# Use datetime module to extract observation date & time and reformat
date = datetime.datetime.strptime(file_id.name.split('_')[-3][1:9], '%Y%m%d').date()
date = date.strftime('%d %b %Y')
# Put extracted/reformated strings together to make image title
title = 'NOAA-20/VIIRS Active Fires I-band  ' + date + ' ' + file_id.name.split('_')[-3][9:11] + ':' + file_id.name.split('_')[-3][11:13] + ' UTC'
# Add plot title
plt.title(title, pad=10, size=8, weight='bold')

# Show plot
plt.show()

# Save figure
# "dpi" is image resolution in dots per inch; use a high dpi (e.g., 1000) for figures w/map image background
# "bbox_inches=tight" sets a "tight" bounding box around saved image
saved_file_format = '.png'
saved_file_name = 'viirs_af_i-band_' + file_id.name.split('_')[-3][1:9] + '_' + file_id.name.split('_')[-3][9:13] + saved_file_format
fig.savefig(saved_file_name, facecolor='w', dpi=1000, bbox_inches='tight')

# Close plot
plt.close()